In [ ]:
from ipynb.fs.full.df_collapser import collapse
import pandas as pd
import glob

In [ ]:
from scipy.signal import savgol_filter

def noise_filter(dataframe):
    for column in dataframe.columns:
        dataframe[column] = savgol_filter(dataframe[column], 10, 2, axis=0)
    return dataframe

In [ ]:
def transform_to_timeseries(dataframe, class_name, sample_number):
    i = dataframe.shape[0]//sample_number
    j=0
    filtered_df=noise_filter(dataframe)
    df_time_series=pd.DataFrame()
    for count in range(1,i):
        print(j)
        print(sample_number*count)
        samples_df=filtered_df.iloc[j:sample_number*count, :]
        new_df=collapse(samples_df)
        if(count==1):
            df_time_series=new_df
        else:
            df_time_series = pd.concat([df_time_series, new_df], axis=0)
        j=sample_number*count
    df_time_series['class']=class_name
    return df_time_series

In [ ]:
def produce_windowed_dataset_csv(secs):
    frequency=50
    sample_number=secs*frequency
    all_folders=glob.glob('A_DeviceMotion_data/*', recursive=True)
    all_data=[]
    class_dict={"dws":"downstairs", "jog": "jogging", "sit":"sit", "std":"standing", "ups":"upstairs", "wlk":"walking"}
    for folder in all_folders:
        all_files=glob.glob(folder+"/*", recursive=True)
        class_name=folder.split('\\')[1].split('_')[0]
        for file in all_files:
            df=pd.read_csv(file, index_col=0)
            # Drop not - measurable columns
            df_clean = df.drop(["attitude.roll", "attitude.pitch", "attitude.yaw", "gravity.y", "gravity.x", "gravity.z"], axis=1)
            temp_df = transform_to_timeseries(df_clean, class_dict[class_name], sample_number)
            all_data.append(temp_df)

    df=pd.concat(all_data, axis=0, ignore_index=True)
    df.to_csv("windowed_DeviceMotion_dataset.csv")

In [ ]:
def produce_windowed_phone_dataset_csv(secs):
    frequency=50
    sample_number=frequency*secs
    all_data=[]
    all_files=glob.glob("honor20readings/*", recursive=True)
    for file in all_files:
        df=pd.read_csv(file)
        class_name=df.iloc[0]["class"]
        df=df.drop("class", axis=1)
        temp_df = transform_to_timeseries(df, class_name, sample_number)
        all_data.append(temp_df)

    df=pd.concat(all_data, axis=0, ignore_index=True)
    df.to_csv("dataset_peaks_honor_3sec.csv")